In [79]:
import pynq
import numpy as np
import time

class Pynqrypt:
    
    def __init__(self, file, post_ap=False):
        self.overlay = pynq.Overlay(file)
        self.ip = self.overlay.pynqrypt_encrypt_0
        self.reg_map = self.ip.register_map
        self.post_ap = post_ap
        if not post_ap:
            self.key_mmio = pynq.MMIO(self.ip.mmio.base_addr + self.reg_map.Memory_key.address, 16 * 8)
            self.nonce_mmio = pynq.MMIO(self.ip.mmio.base_addr + self.reg_map.Memory_nonce.address, 12 * 8)
        self.length = 0
        
    
    def set_key(self, key: bytes):
        if not self.post_ap:
            for i in range(4):
                self.key_mmio.write(4 * i, key[i * 4: (i + 1) * 4])
        else:
            key = int.from_bytes(key, 'big')
            self.reg_map.key_1 = (key) & (2**32 - 1)
            self.reg_map.key_2 = (key >> 32) & (2**32 - 1)
            self.reg_map.key_3 = (key >> 64) & (2**32 - 1)
            self.reg_map.key_4 = (key >> 96) & (2**32 - 1)            

    def set_nonce(self, nonce: bytes):
        if not self.post_ap:
            for i in range(3):
                self.nonce_mmio.write(4 * i, nonce[i * 4: (i + 1) * 4])
        else:
            nonce = int.from_bytes(nonce, 'big')
            self.reg_map.nonce_1 = (nonce) & (2**32 - 1)
            self.reg_map.nonce_2 = (nonce >> 32) & (2**32 - 1)
            self.reg_map.nonce_3 = (nonce >> 64) & (2**32 - 1)
            
    def set_length(self, length: int):
        self.reg_map.plaintext_length_1 = length 
        self.length = length
        
    def get_length(self) -> int:
        return self.reg_map.plaintext_length_1
    
    def get_input_array(self):
        buff = pynq.allocate(self.length, np.uint8)
        self.inbuff = buff
        return buff
    
    def get_output_array(self):
        buff = pynq.allocate(self.length, np.uint8)
        self.outbuff = buff
        return buff
    
    def prepare(self):
        self.inbuff.flush()
        self.reg_map.plaintext_1 = self.inbuff.physical_address
        self.reg_map.ciphertext_1 = self.outbuff.physical_address
    
    def run_blocking(self):
        if not self.reg_map.CTRL.AP_IDLE:
            return
        self.reg_map.CTRL.AP_START = 1
        while not self.reg_map.CTRL.AP_DONE:
            continue
            
    def run_blocking_timed(self):
        if not self.reg_map.CTRL.AP_IDLE:
            return
        
        time_start = time.perf_counter_ns()
        self.reg_map.CTRL.AP_START = 1
        while not self.reg_map.CTRL.AP_DONE:
            continue
        time_stop = time.perf_counter_ns()
        
        return (time_start, time_stop)
        
    def cleanup(self):
        del self.outbuff
        del self.inbuff
        

In [80]:
from Crypto.Cipher import AES
import os
import time

class USBCrypt:
    
    def __init__(self, location='/media/demo/files'):
        self.location = location
        self.key = b'0123456789abcdef'
        self.nonce = b'0123456789ab'
        self.pynqrypt = Pynqrypt(file='./pynqrypt-final-v3.xsa', post_ap=True)
        self.cipher = AES.new(self.key, AES.MODE_CTR, nonce=self.nonce)
        self.pynqrypt.set_key(self.key)
        self.pynqrypt.set_nonce(self.nonce)
        
    def reset_cipher(self):
        self.cipher = AES.new(self.key, AES.MODE_CTR, nonce=self.nonce)    
    
    def decrypt_files(self, accel=True):
        self.encrypt_files(accel, is_encrypt=False)
            
    def encrypt_files(self, accel=True, is_encrypt=True):
        with os.scandir(self.location) as entries:
            for entry in entries:
                start = time.perf_counter_ns()
                
                if accel:
                    self.hw_encrypt_file(entry)
                else:
                    self.sw_encrypt_file(entry)
                    self.reset_cipher()
                
                end = time.perf_counter_ns()
                
                total = (end - start) / 1e9
                
                if is_encrypt:
                    print(f'Encrypted file "{entry.name}" in {total:.3f} seconds')
                else:
                    print(f'Decrypted file "{entry.name}" in {total:.3f} seconds')
    
    def hw_encrypt_file(self, filename):
        data = np.fromfile(filename, np.uint8)
        
        orig_len = len(data)
        
        data = np.pad(data, (0, (16 - orig_len % 16)), 'constant', constant_values=(0, 0))
        
        self.pynqrypt.set_length(len(data))
        
        input_arr = self.pynqrypt.get_input_array()
        
        input_arr[:] = data[:]
        
        out_arr = self.pynqrypt.get_output_array()
        
        self.pynqrypt.prepare()
        self.pynqrypt.run_blocking()
        
        out_arr.invalidate()
        
        with open(filename, 'wb') as f:
            f.write(bytes(out_arr)[:orig_len])
            
        self.pynqrypt.cleanup()
        del data
        del input_arr
        del out_arr
    
    def sw_encrypt_file(self, filename):
        with open(filename, 'rb') as f:
            data = f.read()
            
        data = self.cipher.encrypt(data)
        
        with open(filename, 'wb') as f:
            f.write(data)
            
        del data

In [81]:
usbcrypt = USBCrypt()

In [82]:
usbcrypt.encrypt_files()

Encrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 1.562 seconds
Encrypted file "13 - Minimoog.flac" in 1.071 seconds
Encrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.349 seconds
Encrypted file "IMG_20221224_221202.jpg" in 0.639 seconds
Encrypted file "report.pdf" in 0.029 seconds
Encrypted file "secret password" in 0.005 seconds


In [83]:
usbcrypt.decrypt_files()

Decrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 1.225 seconds
Decrypted file "13 - Minimoog.flac" in 0.888 seconds
Decrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.297 seconds
Decrypted file "IMG_20221224_221202.jpg" in 0.329 seconds
Decrypted file "report.pdf" in 0.030 seconds
Decrypted file "secret password" in 0.006 seconds


In [84]:
usbcrypt.encrypt_files(accel=False)

Encrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 1.946 seconds
Encrypted file "13 - Minimoog.flac" in 1.771 seconds
Encrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.487 seconds
Encrypted file "IMG_20221224_221202.jpg" in 0.874 seconds
Encrypted file "report.pdf" in 0.041 seconds
Encrypted file "secret password" in 0.002 seconds


In [85]:
usbcrypt.decrypt_files(accel=False)

Decrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 2.042 seconds
Decrypted file "13 - Minimoog.flac" in 2.031 seconds
Decrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.477 seconds
Decrypted file "IMG_20221224_221202.jpg" in 0.554 seconds
Decrypted file "report.pdf" in 0.042 seconds
Decrypted file "secret password" in 0.001 seconds


In [86]:
usbcrypt.encrypt_files()

Encrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 1.192 seconds
Encrypted file "13 - Minimoog.flac" in 0.879 seconds
Encrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.293 seconds
Encrypted file "IMG_20221224_221202.jpg" in 0.435 seconds
Encrypted file "report.pdf" in 0.028 seconds
Encrypted file "secret password" in 0.007 seconds


In [87]:
usbcrypt.decrypt_files(accel=False)

Decrypted file "02 - Canzone del maggio (Liberamente tratta da un canto del maggio francese 1968).flac" in 1.935 seconds
Decrypted file "13 - Minimoog.flac" in 1.450 seconds
Decrypted file "Alla Ricerca Del Tempo Perduto.epub" in 0.492 seconds
Decrypted file "IMG_20221224_221202.jpg" in 0.922 seconds
Decrypted file "report.pdf" in 0.041 seconds
Decrypted file "secret password" in 0.001 seconds
